In [1]:
import numpy as np
import qiskit.pulse as pulse
from qiskit_dynamics.pulse.pulse_simulator import PulseSimulator

In [2]:
# this gets us V2 backends
from qiskit_ibm_provider import IBMProvider
#IBMProvider.save_account(token='e138ea6b754f60ffd002db17bcfc3aaf8eb7fd35f51c832aa247c8ffa6ca261b9ca3eb487d3a976b631469141fc693fdb6d37284560395483c47b27dec4a5e42')

provider = IBMProvider()
ibm_backend = provider.get_backend('ibmq_lima')

from qiskit.providers.backend import BackendV2
isinstance(ibm_backend, BackendV2)

True

In [3]:
backend = PulseSimulator.from_backend(ibm_backend, subsystem_list=[0, 1])

In [4]:
from qiskit.pulse import library

amp = 1
sigma = 10
num_samples = 128
#%%
gauss = pulse.library.Gaussian(num_samples, amp, sigma,
                              name="Parametric Gauss")
gauss.draw()


with pulse.build() as schedule:
    pulse.play(gauss, backend.drive_channel(0))
    pulse.play(gauss, backend.drive_channel(1))
    pulse.play(gauss, backend.control_channel([0,1])[0])
    pulse.play(gauss, backend.control_channel([1,0])[0])

schedule.draw()
y0 = np.zeros(backend.solver.model.dim)
y0[0] = 1
t_span = np.array([0, num_samples * backend.solver._dt])
result = backend.run(schedule, y0=y0, t_span=t_span)
#%%
result

Result(backend_name='Pulse Simulator of ibmq_lima', backend_version='0.1', qobj_id='', job_id='', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(unitary=[[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 1.00000000e+00+1.11200898e-11j  7.76571910e-07-1.38821384e-06j
  -3.68859586e-11-1.16395108e-11j ... -1.15648134e-11-4.92083579e-11j
  -2.23361331e-16+2.89313954e-16j  1.30579145e-22-8.63726630e-22j]
 [ 1.00000000e+00+1.27145837e-10j -7.10282616e-07-2.30438137e-06j
  -1.79431785e-10+7.72985221e-11j ...  1.33392100e-11-1.98562396e-10j
   7.07262055e-16+1.07276245e-15j  9.83342785e-21+2.89424477e-20j]
 ...
 [ 7.10872883e-02+4.88502892e-02j  9.35876027e-02-2.67664300e-01j
   6.91343085e-03-8.24361207e-03j ...  3.49236167e-02-1.30597163e-03j
   4.19228823e-02-5.31923150e-02j  1.6725

In [5]:
backend.solver._hamiltonian_channels

['d0', 'd1', 'u0', 'u1', 'u2', 'u3']

In [7]:
from qiskit_dynamics import Solver
import numpy as np

In [22]:
Z = np.array([[1., 0.], [0., -1.]])
X = np.array([[0., 1.], [1., 0.]])

test_solver = Solver(
    static_hamiltonian=2 * X + 3 * Z,
    hamiltonian_operators=[X],
    rotating_frame=Z + X
)

In [23]:
test_solver.model.static_operator

Array([[ 2.+0.j,  1.+0.j],
       [ 1.+0.j, -2.+0.j]], backend='numpy')

In [24]:
test_solver.model.operators

Array([[[ 2.96506192e-17+0.j,  1.00000000e+00+0.j],
        [ 1.00000000e+00+0.j, -7.71461485e-17+0.j]]], backend='numpy')